In [ ]:
# Arnav - computeObjectPoints()
# Output:
# point_cloud - a HxWx3 array with (X,Y,Z) point coordinates at 
# each object pixel. Values of Nan or -1 indicate pixels that 
# are outside of the object mask

# %run "./calibrateCamera.ipynb"
# %run "./computeTwoPlanes.ipynb"
# import numpy as np
# import cv2 

# K, distCoeff = calibrateCamera('./resources_unzipped/AprilBoards.pickle')
# shadow_list =    # shadow plane list of len T, telling shadow plane for every frame T
# front_edge_times = # array in which entry (h,w) tells frame T of pixel (h,w)


In [ ]:
# object_mask_adrs = "resources_unzipped/mask_object.png"
def computeObjectPoints(K, distCoeff, object_mask_adrs, shadow_planes, front_edge_times): 
    # resize object_mask to same dimensions as front_edge_times
    object_mask = cv2.imread(object_mask_adrs, 0)
    scale_percent = 40
    width = int(object_mask.shape[1] * scale_percent / 100)
    height = int(object_mask.shape[0] * scale_percent / 100)
    dim = (width, height)
    object_mask = cv2.resize(object_mask, dim, interpolation = cv2.INTER_AREA)


    # create blank output file
    (H,W) = object_mask.shape
    point_cloud = np.zeros((H,W,3))

    # iterate over every pixel
    for h in range(H):
        for w in range(W):
            if object_mask[h,w] == 0:
                point_cloud[h,w] = np.array([np.NaN, np.NaN, np.NaN])
            else: 
                # backproj valid pixel (h,w) and convert to hom  
                pt = np.array([h,w]).astype(float)          
                d = cv2.undistortPoints(pt, K, distCoeff)
                d = np.squeeze(d)  # remove extraneous size-1 dimension 
                d = np.append(d, [1])   # make hom
            
                # find shadow plane for specific pixel
                Pi_s = shadow_planes[front_edge_times[h,w]]

                # calculate 3d obj point
                n = Pi_s[:-1]  # remove last elem, plane norm vec
                s = Pi_s[-1]   # only last elem, dist from origin
                llambda = -s/(np.dot(n, d))  # distance along ray from img plane to 3d
                obj_point = np.expand_dims(llambda, axis=0)*d  # we now have 3-vec for 3d point on Pi_s
                
                # print(obj_point)
                # add value to point_cloud
                point_cloud[h,w] = obj_point
    
    return point_cloud